In [1]:
import json
import tweepy
import pandas as pd
import numpy as np
import nltk

### Import dataframe of tweets:

In [44]:
data_large = pd.read_csv('tweets_from_tweepy_large.csv')

In [45]:
data_large.head()

,Unnamed: 0,0,Handle,ID,CreatedAt,Location
0,0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,1359905846659059713,2021-02-11 16:43:14,NaN
1,1,RT @rachel_groves: Daily goals: Wake up early....,joscchh,1359905846659059713,2021-02-11 16:43:14,NaN
2,2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,1359101568390414337,2021-02-09 11:27:19,NaN
3,3,RT @EurAsiaBridgeEU: good topic - well done.,joscchh,1359019191358156802,2021-02-09 05:59:59,NaN
4,4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,1359015354828210176,2021-02-09 05:44:44,NaN


In [46]:
data_large.shape

# 10953 tweets extracted from 554 Twitter Followers

(10953, 6)

In [47]:
len(data_large['Handle'])

10953

In [48]:
data_large.drop('Unnamed: 0', axis=1, inplace=True)

In [49]:
# Import dataframe with the Friends Information (including name, handle, location, description, etc)
followers_df = pd.read_csv('twitter_followers_info_df.csv')
followers_df.head()

,Unnamed: 0,Follower,Handle,Location,UserDescription
0,0,Christina Rupp,christinacrupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...
1,1,Johanna Schäfer,joscchh,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
2,2,John Miller,JohnMillerTX,"Fort Worth, TX","#Analytics leader dealing in #machinelearning,..."
3,3,Rosanna Fanni,rosannafanni,NaN,monitoring the European public-political sphere
4,4,Lottie Lane,lottielane_,"Groningen, Nederland",Assistant Prof. and Postdoc at @univgroningen\...


In [50]:
followers_df.shape

(587, 5)

In [51]:
followers_df.drop('Unnamed: 0', axis=1, inplace=True)

### Merge the two dataframes to have Name, Handle, Tweet, and Location in one dataframe.
Merge on `'Handle'` 

In [52]:
df1 = data_large
df2 = followers_df

df = pd.merge(df1, df2, how='left', on='Handle')

In [53]:
df.shape
df.head()

,0,Handle,ID,CreatedAt,Location_x,Follower,Location_y,UserDescription
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,1359905846659059713,2021-02-11 16:43:14,NaN,Christina Rupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...
1,RT @rachel_groves: Daily goals: Wake up early....,joscchh,1359905846659059713,2021-02-11 16:43:14,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,1359101568390414337,2021-02-09 11:27:19,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
3,RT @EurAsiaBridgeEU: good topic - well done.,joscchh,1359019191358156802,2021-02-09 05:59:59,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,1359015354828210176,2021-02-09 05:44:44,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."


In [54]:
df.rename(columns={'0':'TweetText', 'Location_x':'Geocode', 'Location_y':'Location'}, inplace=True)

In [55]:
df.head()

,TweetText,Handle,ID,CreatedAt,Geocode,Follower,Location,UserDescription
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,1359905846659059713,2021-02-11 16:43:14,NaN,Christina Rupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...
1,RT @rachel_groves: Daily goals: Wake up early....,joscchh,1359905846659059713,2021-02-11 16:43:14,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,1359101568390414337,2021-02-09 11:27:19,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
3,RT @EurAsiaBridgeEU: good topic - well done.,joscchh,1359019191358156802,2021-02-09 05:59:59,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."
4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,1359015354828210176,2021-02-09 05:44:44,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e..."


In [56]:
df.shape

(10953, 8)

In [57]:
df.dtypes

# change CreatedAt column to datetime

TweetText           object
Handle              object
ID                   int64
CreatedAt           object
Geocode            float64
Follower            object
Location            object
UserDescription     object
dtype: object

### Cleaning the Location data. 
Make sure all cities/states are coded as country:

In [18]:
locations = df['Location'].unique()

In [19]:
len(locations)

207

In [30]:
# Export locations df as csv, code (0: drop, 1: USA, 2: Germany) in excel, import coded locations as csv and merge with tweets df.

In [22]:
excel = pd.DataFrame(locations)

In [23]:
excel.to_csv('locations_to_code.csv')

In [62]:
### Recode the locations in excel and import the new csv

locations_coded = pd.read_csv('locations_coded.csv')

In [63]:
locations_coded.drop('Unnamed: 0', axis=1, inplace=True)

In [64]:
locations_coded.rename(columns={'0':'Location', '1':'Location_code'}, inplace=True)

In [65]:
locations_coded.head()

,Location,Location_code
0,"Bonn, Deutschland",2
1,"Fort Worth, TX",1
2,NaN,0
3,"Groningen, Nederland",0
4,London,0


In [73]:
# merge tweets df and location codes:

df = df.merge(locations_coded, on='Location', how='left')

In [74]:
df.head()

,TweetText,Handle,ID,CreatedAt,Geocode,Follower,Location,UserDescription,Location_code
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,1359905846659059713,2021-02-11 16:43:14,NaN,Christina Rupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...,2.0
1,RT @rachel_groves: Daily goals: Wake up early....,joscchh,1359905846659059713,2021-02-11 16:43:14,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",2.0
2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,1359101568390414337,2021-02-09 11:27:19,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",2.0
3,RT @EurAsiaBridgeEU: good topic - well done.,joscchh,1359019191358156802,2021-02-09 05:59:59,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",2.0
4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,1359015354828210176,2021-02-09 05:44:44,NaN,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",2.0


In [75]:
# drop ['Location_code']==0

df = df[df['Location_code']!=0.0]

In [76]:
df.shape # see how many columns you are left with after dropping locations not America/Germany

(7123, 9)

In [77]:
df['Location_code'].value_counts()
# 5840 followers from Germany, 1087 from America. 
# Add the small csv to the analysis as well!

2.0    5840
1.0    1087
Name: Location_code, dtype: int64

### Because you only have 1087 tweets from the US, add the small tweets csv to the df and include in analysis as well to get more tweets from American network.

In [78]:
data_small = pd.read_csv('tweets_from_tweepy_small.csv')
data_small.shape

(3892, 6)

In [80]:
data_small.drop('Unnamed: 0', axis=1, inplace=True)

In [83]:
# Import dataframe with the Friends Information (including name, handle, location, description, etc)
friends_df = pd.read_csv('twitter_friends_info_df.csv')

In [84]:
friends_df.drop('Unnamed: 0', axis=1, inplace=True)

In [85]:
# Merge small df and friends:

df1 = data_small
df2 = friends_df

df_small = pd.merge(df1, df2, how='left', on='Handle')

In [87]:
df_small.shape
df_small.head()

,0,Handle,ID,CreatedAt,Location_x,Friend,Location_y,UserDescription
0,Really excited for this event tonight. Always ...,divafeminist,1357386255714115587,2021-02-04 17:51:17,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...
1,@prisonculture YES! Please- we don’t need it. ...,divafeminist,1357069269822033923,2021-02-03 20:51:41,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...
2,@sweetbtchesbrew Such a force. How different i...,divafeminist,1356411441990004738,2021-02-02 01:17:43,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...
3,"Rest In Peace and Power, #KarenLewis. What an ...",divafeminist,1355242388508332036,2021-01-29 19:52:19,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...
4,At the core of US criminal punishment system i...,divafeminist,1355006374313390083,2021-01-29 04:14:29,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...


In [88]:
df_small.rename(columns={'0':'TweetText', 'Location_x':'Geocode', 'Location_y':'Location'}, inplace=True)

In [90]:
locations_small = df_small['Location'].unique()

In [93]:
locations_small = pd.DataFrame(locations_small)
locations_small.to_csv('locations_sm_to_code.csv')

In [94]:
# Export as csv, re-code in excel, import as new csv, merge:

In [95]:
locations_sm_coded = pd.read_csv('locations_sm_coded.csv')
locations_sm_coded.head()

,Unnamed: 0,0,Unnamed: 2
0,0,All Ovah the World,0
1,1,Berlin,2
2,2,"Bonn, Deutschland",2
3,3,"Los Angeles, CA",1
4,4,"Fort Worth, TX",1


In [96]:
locations_sm_coded.drop('Unnamed: 0', axis=1, inplace=True)

In [100]:
locations_sm_coded.rename(columns={'Unnamed: 2': 'Location_code'}, inplace=True)

In [101]:
locations_sm_coded.head()

,Location,Location_code
0,All Ovah the World,0
1,Berlin,2
2,"Bonn, Deutschland",2
3,"Los Angeles, CA",1
4,"Fort Worth, TX",1


In [102]:
df_small = df_small.merge(locations_sm_coded, on='Location', how='left')

In [103]:
df_small.head()

,TweetText,Handle,ID,CreatedAt,Geocode,Friend,Location,UserDescription,Location_code
0,Really excited for this event tonight. Always ...,divafeminist,1357386255714115587,2021-02-04 17:51:17,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...,0.0
1,@prisonculture YES! Please- we don’t need it. ...,divafeminist,1357069269822033923,2021-02-03 20:51:41,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...,0.0
2,@sweetbtchesbrew Such a force. How different i...,divafeminist,1356411441990004738,2021-02-02 01:17:43,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...,0.0
3,"Rest In Peace and Power, #KarenLewis. What an ...",divafeminist,1355242388508332036,2021-01-29 19:52:19,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...,0.0
4,At the core of US criminal punishment system i...,divafeminist,1355006374313390083,2021-01-29 04:14:29,NaN,Treva B,All Ovah the World,Prof @ OSU/feminist/dancer/friend/daughter/aun...,0.0


In [104]:
# Subset for where location_code != 0
df_small = df_small[df_small['Location_code']!=0.0]

In [105]:
df_small.shape

(2707, 9)

In [106]:
df_small['Location_code'].value_counts()
# 1567 German tweets
# 1040 American tweets

2.0    1567
1.0    1040
Name: Location_code, dtype: int64

### Concat df and df_small:
Then make sure to drop duplicate rows!

In [107]:
print("Shape of large df: ",df.shape)
print("Shape of small df: ", df_small.shape)

Shape of large df:  (7123, 9)
Shape of small df:  (2707, 9)


In [108]:
tweets = pd.concat([df, df_small], ignore_index=True)

In [110]:
tweets.shape

(9830, 10)

In [112]:
# Make sure there aren't duplicated rows (for Handles in both friends and followers dataframes for example.)

In [113]:
tweets.drop_duplicates(keep='first', inplace=True)

In [114]:
tweets.shape
# got rid of ~1000 duplicates

(8425, 10)

In [116]:
tweets['Location_code'].value_counts()

# Running your analysis on 6000 tweets from Germany and only 2000 from the US. 

2.0    6002
1.0    2127
Name: Location_code, dtype: int64

### Now that you have all tweets in one df, clean, fix value types, and safe to csv:

In [118]:
tweets.dtypes

TweetText           object
Handle              object
ID                   int64
CreatedAt           object
Geocode            float64
Follower            object
Location            object
UserDescription     object
Location_code      float64
Friend              object
dtype: object

In [119]:
tweets['Country'] = np.where((tweets['Location_code']==2.0), "Germany", "USA") 

In [121]:
tweets['Country'].value_counts() 
# How are these values different than those above?? 
# Don't drop ['Location_code'] just yet. 

Germany    6002
USA        2423
Name: Country, dtype: int64

In [ ]:
# Before dropping Location/UserDescription column, see if there are a lot of Americans that I could manually add to the analysis to get more tweets from the US context!


In [122]:
# save as csv just in case: 
tweets.to_csv('tweets_to_preprocess_NLP.csv')